In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import gzip
import os

2023-03-17 09:37:21.453459: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 09:37:22.112212: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/dist/plumed2/src/lib/:/opt/gromacs/lib:/home/dima/miniconda3/envs/tf/lib/
2023-03-17 09:37:22.112270: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/dist/plumed2/src/lib/:/opt/gromacs/l

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
from tf_model.run_opts import config_runtime
from src.logger import Logger
from tf_model.dataset import read_list, read_item

In [4]:
def save_for_paraview(filename, xyz_tensor, color_tensor, z_shift = 0):
    
    # tosave = torch.cat([xyz_tensor, color_tensor[:,None]], dim=1)
    tosave = np.concatenate((xyz_tensor, color_tensor[:, None]), axis=1)
    # qc = tosave.cpu().detach().numpy()
    qc = tosave
    with open(filename, 'w') as iFile:
        iFile.write("x,y,z,c\n")
        for i in qc:
            x,y,z,c = i
            iFile.write(f'{x},{y},{z+z_shift},{c}\n')
    
    return 0

# Read

In [ ]:
## Parameters

In [5]:
n_cls = 3  # number of classes
# conf_dev = config_runtime['device']
train_f = config_runtime['train_frac']
seed = config_runtime['seed']
learning_rate = config_runtime['learning_rate']
batch_size = config_runtime['batch_size']
hid_size = config_runtime['hidden_size']
log_step = config_runtime['log_step']
epochs = config_runtime['num_epochs']
run_name = config_runtime['run_name']

In [6]:
hid_size

32

## Protein

In [24]:
# protein = 1WVC_A 7ZX5_X
pid = "1WVC"
cid = "A"
p, x, y = read_item(pid, cid)
n_input = x.shape[1]

# Model

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(n_input, activation='relu'),
    tf.keras.layers.Dense(hid_size, activation='relu'),
    tf.keras.layers.Dropout(0.10),
    tf.keras.layers.Dense(hid_size, activation='relu'),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(hid_size, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(n_cls)
])

In [26]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.load_weights("./tf_model/model/model.ckpt")



# Predict

In [27]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [28]:
predictions = probability_model(x)
y_pred = np.argmax(predictions, axis=1)

In [32]:
y_pred[320:490]

array([1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2,
       2, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 2, 1, 2, 0, 0, 0, 0, 1, 1, 2, 2,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1])

In [31]:
y[320:490]

array([1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2,
       2, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])

In [33]:
save_for_paraview(f"predicted_{pid}_{cid}.csv", p, y_pred)
save_for_paraview(f"real_{pid}_{cid}.csv", p, y)

0